In [21]:
import tensorflow as tf
import numpy as np
import pandas as pd
import ast  
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from tensorflow.keras.preprocessing.sequence import pad_sequences
import seaborn as sns
from keras.models import model_from_yaml
import numpy
import os
from google.colab import drive
drive.mount('/content/drive/')  

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [0]:
df = pd.read_csv('drive/My Drive/Tesis/Data/Flujo1.csv', sep=";")
#df = pd.read_csv('../Data/Flujo1.csv', sep=";")
df['Respuesta'] = df['Respuesta'].apply(ast.literal_eval)
def enumerate_dimensions(dimension, list_dimensions): 
    return list_dimensions.index(dimension)
def wordsToNumbers(tokens, vocabulary):
    number_array = []
    for i in tokens:
        number_array.append(vocabulary.index(i)+1)
    return np.asarray(number_array)    

In [0]:
# Se Construye Dataframe solo de Social Externo
df = df[df.Area == 'Social Interno']
# Lista de dimensiones se pasa a numeros
list_dimensions = df.Dimension.unique().tolist()
num_classes = len(list_dimensions)
df['_dimension'] = df['Dimension'].apply(enumerate_dimensions, list_dimensions = list_dimensions)
#Generamos un vocabulario de palabras
vocabulary = []
df['Respuesta'].apply(vocabulary.append)
vocabulary = [item for sublist in vocabulary for item in sublist]
vocabulary = list(set(vocabulary))

In [0]:
df['NumRespuesta'] = df.Respuesta.apply (wordsToNumbers, vocabulary = vocabulary)
# Rellenamos las matrices con 0 para que todas tengan el mismo tamaño
X_completo = pad_sequences(df.NumRespuesta, maxlen=10, dtype='object', padding='post', value = 0)
# Pasamos las dimensiones a np array
y_completo = np.array(df['_dimension'])
X, test_x, y, test_y = train_test_split(X_completo, y_completo, test_size = 0.2, random_state = 0)
resultados = []

In [29]:
vocab_len= len(vocabulary)+1
num_classes = len(list_dimensions) #9
ouput_dim_embedding = 100
lstm_units = 200
ouputs_dense = num_classes
epoch = 7
batch_size = 100
optimizer = 'adam'

model = tf.keras.Sequential()
#Se agrega capa embedding que hace w2v
model.add(tf.keras.layers.Embedding(input_dim=vocab_len, output_dim = ouput_dim_embedding, input_shape=(X_completo.shape[1],)))
#model.add(tf.keras.layers.SpatialDropout1D(0.4))
model.add(tf.keras.layers.LSTM(units=lstm_units, activation='sigmoid', dropout=0.7, recurrent_dropout=0.7))
model.add(tf.keras.layers.Dense(units=ouputs_dense, activation='sigmoid'))
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])
history = model.fit(X, y, epochs = epoch, batch_size=batch_size)
print()
print('Resultados con datos de testeo ')
print()
test_loss, test_acurracy = model.evaluate(test_x, test_y)
print()
print('Data Test accuracy: {}'.format(test_acurracy))
predictions = model.predict_classes(test_x)
resultados.append(test_acurracy)
#Matriz de confusion
confusion = tf.confusion_matrix(labels=test_y, predictions= predictions, dtype=tf.dtypes.int32, num_classes=num_classes)
print('-Matriz de confusion')
print(confusion.eval(session=tf.Session()))
resultados


Train on 41073 samples
Epoch 1/7
41073/41073 [==============================] - 9s 211us/sample - loss: 2.0185 - sparse_categorical_accuracy: 0.2403
Epoch 2/7
41073/41073 [==============================] - 8s 194us/sample - loss: 1.1987 - sparse_categorical_accuracy: 0.6281
Epoch 3/7
41073/41073 [==============================] - 8s 191us/sample - loss: 0.8866 - sparse_categorical_accuracy: 0.7326
Epoch 4/7
41073/41073 [==============================] - 8s 191us/sample - loss: 0.7618 - sparse_categorical_accuracy: 0.7712
Epoch 5/7
41073/41073 [==============================] - 8s 194us/sample - loss: 0.6896 - sparse_categorical_accuracy: 0.7930
Epoch 6/7
41073/41073 [==============================] - 8s 194us/sample - loss: 0.6337 - sparse_categorical_accuracy: 0.8088
Epoch 7/7
41073/41073 [==============================] - 8s 191us/sample - loss: 0.5895 - sparse_categorical_accuracy: 0.8252

Resultados con datos de testeo 

10269/10269 [==============================] - 2s 203us/sampl

[0.75800955, 0.7533353, 0.7598598, 0.75839907, 0.7559646]

In [30]:
model_yaml = model.to_yaml()
with open("drive/My Drive/Tesis/RNN/SINT/modelSINT.yaml", "w") as yaml_file:
    yaml_file.write(model_yaml)
# serialize weights to HDF5
model.save_weights("drive/My Drive/Tesis/RNN/SINT/modelSINT.h5")
print("Saved model to disk")

Saved model to disk


In [0]:
vocab_len= len(vocabulary)+1
num_classes = len(list_dimensions) #9
ouput_dim_embedding = 60
lstm_units = 100
ouputs_dense = num_classes
epoch = 7
batch_size = 60
optimizer = 'adam'

kf = KFold(n_splits = 10, shuffle = True, random_state = 2)
resultados = []
contador = 1
for valores_entrenamiento, valores_testeo in kf.split(X_completo):
    model = tf.keras.Sequential()
    #Se agrega capa embedding que hace w2v
    model.add(tf.keras.layers.Embedding(input_dim=vocab_len, output_dim = ouput_dim_embedding, input_shape=(X_completo.shape[1],)))
    model.add(tf.keras.layers.LSTM(units=lstm_units, activation='sigmoid'))
    model.add(tf.keras.layers.Dense(units=ouputs_dense, activation='sigmoid'))
    model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])
    history = model.fit(X_completo[valores_entrenamiento], y_completo[valores_entrenamiento], epochs = epoch, batch_size = batch_size)
    print()
    print(contador, '-Resultados con datos de testeo ')
    print()
    test_loss, test_acurracy = model.evaluate(X_completo[valores_testeo], y_completo[valores_testeo])
    print()
    print(contador, '-Data Test accuracy: {}'.format(test_acurracy))
    resultados.append(test_acurracy)
    print()
    predictions = model.predict_classes(X_completo[valores_testeo])
    #Matriz de confusion
    confusion = tf.confusion_matrix(labels=y_completo[valores_testeo], predictions= predictions, dtype=tf.dtypes.int32, num_classes=num_classes)
    print(contador, '-Matriz de confusion')
    print(confusion.eval(session=tf.Session()))
    print()
    contador = contador + 1

Train on 46207 samples
Epoch 1/7
46207/46207 [==============================] - 20s 427us/sample - loss: 1.7527 - sparse_categorical_accuracy: 0.3927
Epoch 2/7
46207/46207 [==============================] - 19s 415us/sample - loss: 1.0083 - sparse_categorical_accuracy: 0.6928
Epoch 3/7
46207/46207 [==============================] - 19s 412us/sample - loss: 0.8484 - sparse_categorical_accuracy: 0.7470
Epoch 4/7
46207/46207 [==============================] - 19s 406us/sample - loss: 0.7774 - sparse_categorical_accuracy: 0.7666
Epoch 5/7
46207/46207 [==============================] - 19s 402us/sample - loss: 0.7315 - sparse_categorical_accuracy: 0.7782
Epoch 6/7
46207/46207 [==============================] - 19s 409us/sample - loss: 0.6973 - sparse_categorical_accuracy: 0.7868
Epoch 7/7
46207/46207 [==============================] - 19s 403us/sample - loss: 0.6736 - sparse_categorical_accuracy: 0.7930

1 -Resultados con datos de testeo 

5135/5135 [==============================] - 1s 267

NameError: ignored